In [25]:
import itertools, json, math
from pathlib import Path
from typing import List, Set, Union

import networkx as nx
from pulp import (
    LpBinary,
    LpMaximize,
    LpProblem,
    LpStatus,
    LpVariable,
    PULP_CBC_CMD,
    lpSum,
)

# ---------------------------------------------------------------
# 1) LETTURA JSON
# ---------------------------------------------------------------
def load_circles(json_path:str, key="circles"):
    data = json.loads(Path(json_path).read_text())
    centers = [tuple(c["center"]) for c in data["circles"]]

    nodes = []
    for idx, c in enumerate(data[key]):
        if "center" in c:
            x, y = c["center"]
        else:
            x, y = c["x"], c["y"]
        nodes.append({"id": c.get("id", idx), "x": float(x), "y": float(y)})
    return nodes



# ---------------------------------------------------------------
# 2) GRAFO NON PESATO
# ---------------------------------------------------------------
def circles_to_graph(circles, r=18.0, tol=1.0):
    """due nodi adiacenti se |dist - 2r| <= tol"""
    target = 2 * r
    G = nx.Graph()
    for c in circles:
        G.add_node(c["id"], **c)

    for a, b in itertools.combinations(circles, 2):
        if abs(math.dist((a["x"], a["y"]), (b["x"], b["y"])) - target) <= tol:
            G.add_edge(a["id"], b["id"])
    return G


# ---------------------------------------------------------------
# 3-bis) GREEDY per il seed
# ---------------------------------------------------------------
def greedy_seed(G: nx.Graph, S: int, P: int) -> List[Set[int]]:
    comps, free = [], set(G.nodes)
    for _ in range(S):
        root = max(free, key=lambda v: G.degree(v))
        comp, front = {root}, [root]
        while len(comp) < P and front:
            v = front.pop(0)
            for u in G.neighbors(v):
                if u in free and u not in comp:
                    comp.add(u)
                    front.append(u)
                if len(comp) == P:
                    break
        # riempi se mancano nodi (caso rado, ma sicuro)
        if len(comp) < P:
            comp.update(list(free - comp)[: P - len(comp)])
        comps.append(comp)
        free -= comp
    if any(len(c) < P for c in comps):
        raise ValueError("Greedy fallita: partizionamento incompleto")
    return comps


# ---------------------------------------------------------------
# 3) ILP con warm-start
# ---------------------------------------------------------------


def dense_connected_partition(G: nx.Graph,
                              S: int,                 # nº componenti
                              P: int,                 # nodi per componente
                              *,
                              seed=None,              # lista di insiemi
                              max_seconds=60,
                              frac_gap=0.05):
    """
    Partiziona il grafo G in S componenti connesse da P nodi massimizzando
    il nº di archi interni.  Restituisce (componenti, status CBC).

    * seed        – lista [ set(v_i) ] (facoltativa) per warm-start
    * max_seconds – tempo-limite in secondi
    * frac_gap    – gap relativo ammesso (0‥1)
    """
    # ---------- modello ----------
    prob = LpProblem("DensePartition", LpMaximize)

    # variabili x (nodo in componente)
    x = {(v, c): LpVariable(f"x_{v}_{c}", 0, 1, LpBinary)
         for v in G.nodes for c in range(S)}
    # variabili y (arco interno)
    y = {(u, v, c): LpVariable(f"y_{u}_{v}_{c}", 0, 1, LpBinary)
         for u, v in G.edges for c in range(S)}
    # flusso orientato su entrambi i versi
    f = {(u, v, c): LpVariable(f"f_{u}_{v}_{c}", 0)
         for u, v in G.edges for c in range(S)}
    f.update({(v, u, c): LpVariable(f"f_{v}_{u}_{c}", 0)
              for u, v in G.edges for c in range(S)})
    # radici scelte dal solver
    r = {(v, c): LpVariable(f"root_{v}_{c}", 0, 1, LpBinary)
         for v in G.nodes for c in range(S)}

    # ---------- warm-start ----------
    if seed:
        for c, comp in enumerate(seed):
            for v in comp:                 # suggerisce la partizione
                x[v, c].setInitialValue(1)
            root = min(comp)               # radice candidata (facolt.)
            r[root, c].setInitialValue(1)
    # ---------------------------------

    # ---------- vincoli ----------
    # (1) esattamente P nodi per componente
    for c in range(S):
        prob += lpSum(x[v, c] for v in G.nodes) == P

    # (2) ogni nodo in una sola componente
    for v in G.nodes:
        prob += lpSum(x[v, c] for c in range(S)) == 1

    # (3) coerenza arco–nodi
    for (u, v) in G.edges:
        for c in range(S):
            prob += y[u, v, c] <= x[u, c]
            prob += y[u, v, c] <= x[v, c]

    # (4) bilancio di flusso per garantire connettività
    for c in range(S):
        for v in G.nodes:
            infl  = lpSum(f[u, v, c] for u in G.neighbors(v))
            outfl = lpSum(f[v, u, c] for u in G.neighbors(v))
            prob += outfl - infl == P * r[v, c] - x[v, c]

        for (u, v) in G.edges:
            prob += f[u, v, c] <= (P - 1) * y[u, v, c]
            prob += f[v, u, c] <= (P - 1) * y[u, v, c]

        # (4b) una sola radice per componente
        prob += lpSum(r[v, c] for v in G.nodes) == 1

    # ---------- obiettivo ----------
    prob += lpSum(y[u, v, c] for (u, v) in G.edges for c in range(S))

    # ---------- risoluzione ----------
    solver = PULP_CBC_CMD(
        msg=True,
        timeLimit=max_seconds,
        gapRel=frac_gap,
        options=["heur=on", "fpump=on"]
    )
    prob.solve(solver)

    status = LpStatus[prob.status]          # 'Optimal', 'Feasible', …

    if status not in ("Optimal", "Feasible"):
        raise RuntimeError(f"Solver exit: {status}")

    # componenti come liste di nodi
    comps = [
        [v for v in G.nodes if x[v, c].value() > 0.5]
        for c in range(S)
    ]
    return comps, status


# ---------------------------------------------------------------
# 4) MAIN
# ---------------------------------------------------------------
if __name__ == "__main__":
    S, P = 5, 25
    circles = load_circles("my_circles.json")
    G = circles_to_graph(circles, r=18.0, tol=1.0)

    print(f"Nodi: {len(G)}   Archi: {G.number_of_edges()}")
    print(f"Grado minimo: {min(dict(G.degree()).values())}")

    # seed greedy
    seed = greedy_seed(G, S, P)
    

    comps, status = dense_connected_partition(
        G, S, P, seed=seed, max_seconds=60, frac_gap=0.05
    )
    print(f"Solver status: {status}")
    for i, c in enumerate(comps, 1):
        print(f"Comp {i}: {len(c)} nodi  ->  {c[:8]}{' …' if len(c)>8 else ''}")

Nodi: 125   Archi: 327
Grado minimo: 2
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/tirocinio_IA/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/20/448h2f1933x4jpn5p3l2lb7c0000gn/T/fe2a3a459c1d4892bf44f712fb9c821f-pulp.mps -max -sec 60 -heur=on -fpump=on -ratio 0.05 -timeMode elapsed -branch -printingOptions all -solution /var/folders/20/448h2f1933x4jpn5p3l2lb7c0000gn/T/fe2a3a459c1d4892bf44f712fb9c821f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7305 COLUMNS
At line 37456 RHS
At line 44757 BOUNDS
At line 47643 ENDATA
Problem MODEL has 7300 rows, 6155 columns and 22745 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 60
No match for fpump - ? for list of commands
ratioGap was changed from 0 to 0.05
Option for timeMode changed from cpu to elapsed
Continuous objective value is 327 - 1.34 seconds
Cgl0004I processed model has 

RuntimeError: Solver exit: Not Solved

In [18]:
import pulp
print(pulp.__version__)     # vedrai 1.x

3.0.2


In [27]:
import itertools, json, math
from pathlib import Path
from typing import List, Set, Union

import networkx as nx
from pulp import (
    LpBinary,
    LpMaximize,
    LpProblem,
    LpStatus,
    LpVariable,
    PULP_CBC_CMD,
    lpSum,
)

# ---------------------------------------------------------------
# 1) LETTURA JSON
# ---------------------------------------------------------------
def load_circles(json_path:str, key="circles"):
    data = json.loads(Path(json_path).read_text())
    centers = [tuple(c["center"]) for c in data["circles"]]

    nodes = []
    for idx, c in enumerate(data[key]):
        if "center" in c:
            x, y = c["center"]
        else:
            x, y = c["x"], c["y"]
        nodes.append({"id": c.get("id", idx), "x": float(x), "y": float(y)})
    return nodes



# ---------------------------------------------------------------
# 2) GRAFO NON PESATO
# ---------------------------------------------------------------
def circles_to_graph(circles, r=18.0, tol=1.0):
    """due nodi adiacenti se |dist - 2r| <= tol"""
    target = 2 * r
    G = nx.Graph()
    for c in circles:
        G.add_node(c["id"], **c)

    for a, b in itertools.combinations(circles, 2):
        if abs(math.dist((a["x"], a["y"]), (b["x"], b["y"])) - target) <= tol:
            G.add_edge(a["id"], b["id"])
    return G


# ---------------------------------------------------------------
# 3-bis) GREEDY per il seed
# ---------------------------------------------------------------
def greedy_seed(G: nx.Graph, S: int, P: int) -> List[Set[int]]:
    comps, free = [], set(G.nodes)
    for _ in range(S):
        root = max(free, key=lambda v: G.degree(v))
        comp, front = {root}, [root]
        while len(comp) < P and front:
            v = front.pop(0)
            for u in G.neighbors(v):
                if u in free and u not in comp:
                    comp.add(u)
                    front.append(u)
                if len(comp) == P:
                    break
        # riempi se mancano nodi (caso rado, ma sicuro)
        if len(comp) < P:
            comp.update(list(free - comp)[: P - len(comp)])
        comps.append(comp)
        free -= comp
    if any(len(c) < P for c in comps):
        raise ValueError("Greedy fallita: partizionamento incompleto")
    return comps


# ---------------------------------------------------------------
# 3) ILP con warm-start
# ---------------------------------------------------------------


import itertools
from typing import List, Tuple, Dict
import networkx as nx
from pulp import LpBinary, LpMaximize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD, LpStatus


def dense_connected_partition(G: nx.Graph, S: int, P: int, seed: List[List[int]] = None, max_seconds: int = 300, frac_gap: float = 0.05) -> Tuple[List[List[int]], str]:
    prob = LpProblem("DensePartition", LpMaximize)

    # Variabili di partizione
    x = {(v, c): LpVariable(f"x_{v}_{c}", cat=LpBinary) for v in G.nodes for c in range(S)}
    y = {(u, v, c): LpVariable(f"y_{u}_{v}_{c}", cat=LpBinary) for u, v in G.edges for c in range(S)}
    y.update({(v, u, c): LpVariable(f"y_{v}_{u}_{c}", cat=LpBinary) for u, v in G.edges for c in range(S)})
    f = {(u, v, c): LpVariable(f"f_{u}_{v}_{c}", lowBound=0) for u, v in G.edges for c in range(S)}
    f.update({(v, u, c): LpVariable(f"f_{v}_{u}_{c}", lowBound=0) for u, v in G.edges for c in range(S)})
    rvar = {(v, c): LpVariable(f"root_{v}_{c}", cat=LpBinary) for v in G.nodes for c in range(S)}

    # Inizializzazione con il seed
    # ---- Warm-start con soluzione greedy --------------------------------------
    if seed:
        for c, comp in enumerate(seed):
            comp_list = list(comp)  # Converti il set in una lista
            for v in comp_list:
                x[v, c].setInitialValue(1)
                rvar[v, c].setInitialValue(1 if v == comp_list[0] else 0)

    # Vincoli per la connettività
    for c in range(S):
        # Ogni partizione ha esattamente P nodi
        prob += lpSum(x[v, c] for v in G.nodes) == P

        # Una sola radice per componente
        prob += lpSum(rvar[v, c] for v in G.nodes) == 1

        # Radice deve essere parte della componente
        for v in G.nodes:
            prob += rvar[v, c] <= x[v, c]

        # Flusso di connessione
        for v in G.nodes:
            in_flow = lpSum(f[u, v, c] for u in G.neighbors(v))
            out_flow = lpSum(f[v, u, c] for u in G.neighbors(v))
            prob += out_flow - in_flow == P * rvar[v, c] - x[v, c]

        # Coerenza arco-nodo
        for (u, v) in G.edges:
            prob += y[u, v, c] <= x[u, c]
            prob += y[u, v, c] <= x[v, c]
            prob += f[u, v, c] <= (P - 1) * y[u, v, c]
            prob += f[v, u, c] <= (P - 1) * y[v, u, c]

    # Vincoli di partizione (ogni nodo in una sola componente)
    for v in G.nodes:
        prob += lpSum(x[v, c] for c in range(S)) == 1

    # Funzione obiettivo: massimizza archi interni
    prob += lpSum(y[u, v, c] for (u, v) in G.edges for c in range(S))

    # Parametri del solver
    solver = PULP_CBC_CMD(
        msg=True,
        options=[
            "sec", str(max_seconds),
            "ratio", str(frac_gap),
            "heur", "on",
            "fpump", "on"
        ]
    )

    # Risoluzione del problema
    prob.solve(solver)
    status = LpStatus[prob.status]

    if status not in ("Optimal", "Feasible"):
        raise RuntimeError(f"Solver exit: {status}")

    # Estrai le componenti
    components = [[v for v in G.nodes if x[v, c].value() > 0.5] for c in range(S)]
    return components, status


# ---------------------------------------------------------------
# 4) MAIN
# ---------------------------------------------------------------
if __name__ == "__main__":
    S, P = 5, 25
    circles = load_circles("my_circles.json")
    G = circles_to_graph(circles, r=18.0, tol=1.0)

    print(f"Nodi: {len(G)}   Archi: {G.number_of_edges()}")
    print(f"Grado minimo: {min(dict(G.degree()).values())}")

    # seed greedy
    seed = greedy_seed(G, S, P)
    

    comps, status = dense_connected_partition(
        G, S, P, seed=seed, max_seconds=60, frac_gap=0.05
    )
    print(f"Solver status: {status}")
    for i, c in enumerate(comps, 1):
        print(f"Comp {i}: {len(c)} nodi  ->  {c[:8]}{' …' if len(c)>8 else ''}")

Nodi: 125   Archi: 327
Grado minimo: 2
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/tirocinio_IA/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/20/448h2f1933x4jpn5p3l2lb7c0000gn/T/f6cfc9d1f478481ca6faffd60ee31276-pulp.mps -max -sec -60 -ratio -0.05 -heur -on -fpump -on -timeMode elapsed -branch -printingOptions all -solution /var/folders/20/448h2f1933x4jpn5p3l2lb7c0000gn/T/f6cfc9d1f478481ca6faffd60ee31276-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7930 COLUMNS
At line 42601 RHS
At line 50527 BOUNDS
At line 55048 ENDATA
Problem MODEL has 7925 rows, 7790 columns and 23995 elements
Coin0008I MODEL read with 0 errors
-60 was provided for seconds - valid range is -1 to 1.79769e+308
-0.05 was provided for ratioGap - valid range is 0 to 1.79769e+308
heuristicsOnOff has value on
No match for on - ? for list of commands
No match for fpump - ? for list of com

KeyboardInterrupt: 

Option for timeMode changed from cpu to elapsed
Continuous objective value is 327 - 0.67 seconds
Cgl0004I processed model has 6290 rows, 6155 columns (2885 integer (2885 of which binary)) and 20725 elements
Cbc0038I Initial state - 2646 integers unsatisfied sum - 457
Cbc0038I Pass   1: (1.15 seconds) suminf.    6.42333 (48) obj. 0 iterations 4552
Cbc0038I Pass   2: (1.17 seconds) suminf.    3.77333 (30) obj. -0.333333 iterations 285
Cbc0038I Pass   3: (1.18 seconds) suminf.    2.28833 (17) obj. -1.20833 iterations 310
Cbc0038I Pass   4: (1.19 seconds) suminf.    2.24667 (20) obj. -1.16667 iterations 56
Cbc0038I Pass   5: (1.20 seconds) suminf.    1.57167 (10) obj. -4.29167 iterations 410
Cbc0038I Pass   6: (1.21 seconds) suminf.    1.25167 (10) obj. -4.29167 iterations 77
Cbc0038I Pass   7: (1.22 seconds) suminf.    0.48000 (4) obj. -7 iterations 189
Cbc0038I Pass   8: (1.23 seconds) suminf.    0.16000 (4) obj. -7 iterations 68
Cbc0038I Pass   9: (1.24 seconds) suminf.    1.28000 (7) o